In [30]:
import numpy as np
import pandas as pd
import time
from amc_parser import *
from T3Dviewer import *
import os
import sys
from pathlib import Path

root:
    Order - Values [TX TY TZ : Translation, RX RY RZ: rotation] Channels of motion on VICON
    
Data information:
    root:
    order, axis, position, orientation
    bonedata:
    id, name, direction, length, axis, dof, limits (?), parent, child

end goal:
get table of global position for each part ([x,y,z]-data)(bone-column)(time-row)

In [31]:
#abs_path = os.getcwd()
asf_path = 'data\\01.asf'
amc_path = 'data\\01_01.amc'
joints = parse_asf(asf_path)
motions = parse_amc(amc_path)


In [26]:
a=list(joints.keys())
MCapVal = pd.DataFrame(data=None,columns=a)

#for j in joints.values():
    #j.name
    #(j.__dict__.keys())
    

Series([], Name: root, dtype: object)

In [40]:
#motion is a list -> dict
motions[0]['root']
#np.reshape(np.array(motions[0]['root'][:3]), [3, 1])

[9.37216, 17.8693, -17.3198, -2.01677, -7.59696, -3.23164]

In [62]:
#joints['root'].set_motion(motions[1])
loopList = range(len(motions))

for i in loopList[1:]:
    print (i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


2388
2389
2390
2391
2392
2393
2394
2395
2396
2397
2398
2399
2400
2401
2402
2403
2404
2405
2406
2407
2408
2409
2410
2411
2412
2413
2414
2415
2416
2417
2418
2419
2420
2421
2422
2423
2424
2425
2426
2427
2428
2429
2430
2431
2432
2433
2434
2435
2436
2437
2438
2439
2440
2441
2442
2443
2444
2445
2446
2447
2448
2449
2450
2451
2452
2453
2454
2455
2456
2457
2458
2459
2460
2461
2462
2463
2464
2465
2466
2467
2468
2469
2470
2471
2472
2473
2474
2475
2476
2477
2478
2479
2480
2481
2482
2483
2484
2485
2486
2487
2488
2489
2490
2491
2492
2493
2494
2495
2496
2497
2498
2499
2500
2501
2502
2503
2504
2505
2506
2507
2508
2509
2510
2511
2512
2513
2514
2515
2516
2517
2518
2519
2520
2521
2522
2523
2524
2525
2526
2527
2528
2529
2530
2531
2532
2533
2534
2535
2536
2537
2538
2539
2540
2541
2542
2543
2544
2545
2546
2547
2548
2549
2550
2551
2552
2553
2554
2555
2556
2557
2558
2559
2560
2561
2562
2563
2564
2565
2566
2567
2568
2569
2570
2571
2572
2573
2574
2575
2576
2577
2578
2579
2580
2581
2582
2583
2584
2585
2586
2587


In [74]:
a=[]
b=list(joints.keys())
MCapVal = pd.DataFrame(data=None,columns=b)
loopList = range(len(motions))

for i in loopList[1:]:
    joints['root'].set_motion(motions[i])
    for j in joints.values():
        a.append(j.coordinate)
        #print(j.name)
        #MCapVal[j.name].append(j.coordinate)
    #print(a)
    MCapVal.loc[len(MCapVal), :] = a
    a=[]
    #break
MCapVal
#MCapVal.append(a)

,root,lhipjoint,lfemur,ltibia,lfoot,ltoes,rhipjoint,rfemur,rtibia,rfoot,...,lhand,lfingers,lthumb,rclavicle,rhumerus,rradius,rwrist,rhand,rfingers,rthumb
0,"[[9.37285], [17.8666], [-17.3192]]","[[10.507490027271157], [16.03930756343807], [-...","[[10.884166060132301], [8.893104724450117], [-...","[[11.525169700826622], [1.934066905518061], [-...","[[11.584778016069393], [1.283455933768177], [-...","[[11.585572218180584], [1.2731913244161257], [...","[[7.866159319895818], [16.182398538376212], [-...","[[7.73159136479609], [8.759732304322192], [-16...","[[7.778478392082155], [1.8749755638183263], [-...","[[7.23754191116742], [1.0800932966018628], [-1...",...,"[[11.453055747013188], [27.457415484999085], [...","[[11.417805818922428], [27.969878726939218], [...","[[11.10181074557782], [27.37597924576794], [-1...","[[6.383380840765562], [25.228223029426932], [-...","[[6.3754424927214846], [19.990985294385965], [...","[[5.875758063108678], [16.8336709865629], [-16...","[[5.625915122896883], [15.25500924909225], [-1...","[[5.358981732808545], [14.692623410916266], [-...","[[5.201548767021075], [14.216056866084873], [-...","[[5.62827020582573], [14.563584835604978], [-1..."
1,"[[9.36924], [17.8658], [-17.3215]]","[[10.501334187830546], [16.036663347679298], [...","[[10.882963529272688], [8.8907934875464], [-16...","[[11.527224547540587], [1.9305524196377402], [...","[[11.590310880156249], [1.28068359633963], [-1...","[[11.591186923636236], [1.2861021683655778], [...","[[7.859955708251235], [16.184779591363995], [-...","[[7.735315447581611], [8.76206321332236], [-16...","[[7.784320443053972], [1.881098157575316], [-1...","[[7.240805919074459], [1.0840930724686795], [-...",...,"[[11.434770522215365], [27.46051320246637], [-...","[[11.40205115058915], [27.968273854774804], [-...","[[11.08759110103984], [27.373987214437804], [-...","[[6.38148193356199], [25.22932149094934], [-18...","[[6.379575762701726], [19.992008319363464], [-...","[[5.894840769205443], [16.82929172367514], [-1...","[[5.652472568754472], [15.24792883442973], [-1...","[[5.382927175469213], [14.686788726237884], [-...","[[5.223230843389892], [14.210979567069476], [-...","[[5.649346609982229], [14.55766492159211], [-1..."
2,"[[9.37257], [17.8597], [-17.326]]","[[10.515237228269589], [16.0372504392628], [-1...","[[10.891291434277925], [8.891147970130026], [-...","[[11.532934124024788], [1.9305919538086327], [...","[[11.59530920559086], [1.2826730336951617], [-...","[[11.59528379250497], [1.2964076285238622], [-...","[[7.872310228952777], [16.17372988570562], [-1...","[[7.7413095303540205], [8.750976462570696], [-...","[[7.785346168900364], [1.8746486144425685], [-...","[[7.241471756686602], [1.0802561910504598], [-...",...,"[[11.420870113065089], [27.448939390219415], [...","[[11.391107541240308], [27.950586380875585], [...","[[11.076941841624393], [27.35702106566765], [-...","[[6.384582487848663], [25.220032525317347], [-...","[[6.3876009292845675], [19.983022668274536], [...","[[5.903286442694437], [16.809695201811223], [-...","[[5.661128496306877], [15.223026861773548], [-...","[[5.389455524826781], [14.662945168243274], [-...","[[5.227987543376143], [14.187730319462956], [-...","[[5.656859766501487], [14.530770406722453], [-..."
3,"[[9.37326], [17.8536], [-17.3317]]","[[10.517716432145379], [16.033134809867732], [...","[[10.891420858408436], [8.886827285573752], [-...","[[11.534955241275735], [1.926500048790576], [-...","[[11.59318067548555], [1.2822791262258841], [-...","[[11.592014853643933], [1.288089065017532], [-...","[[7.874500956939514], [16.16754217239844], [-1...","[[7.744092167548611], [8.744818128584987], [-1...","[[7.786978665109136], [1.8694308609536776], [-...","[[7.239688453374954], [1.0757981154243128], [-...",...,"[[11.41616958409227], [27.451938074775303], [-...","[[11.387173424753309], [27.955860837493127], [...","[[11.0678983935498], [27.365126560770733], [-1...","[[6.384006849960199], [25.209248668197922], [-...","[[6.390683414444546], [19.97213895

In [73]:
#len(a)
a
#MCapVal
#joints['ltibia'].__dict__

array([[  5.62827021],
       [ 14.56358484],
       [-16.10480728]])

In [156]:
a=joints['head'].coordinate

In [161]:
euler2mat(*a)
#euler2mat(*a)

array([[ 0.50629016,  0.51648238, -0.69059121],
       [-0.49517815,  0.8297462 ,  0.2575264 ],
       [ 0.70602328,  0.2115826 ,  0.67584313]])

In [10]:
with open('./data/01.asf') as f:
    content = f.read().splitlines()

In [25]:
for c, value in enumerate(content, 0):
    if 'begin' in value:
        print (c)

17
24
35
44
54
63
70
81
90
100
109
120
131
142
153
164
175
185
196
205
214
224
233
243
253
264
273
282
292
301
312
